In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml
from typing import Union

In [2]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
# import data_generator.fraud.time
# import data_generator.fraud.txndata
# import data_generator.configs
# import data_generator.utils
# importlib.reload(data_generator.fraud.time)
# importlib.reload(data_generator.fraud.txndata)
# importlib.reload(data_generator.configs)
# importlib.reload(data_generator.utils)
from data_generator.general_time import *
from data_generator.utils import build_transaction, create_txns_df, sample_category, get_values_from_truncnorm
from data_generator.fraud.txndata import FraudTxnPartData
from data_generator.fraud.time import derive_from_last_time
from data_generator.fraud.drops.behavior import PurchBehaviorHandler
from data_generator.fraud.drops.build.config import DropConfigBuilder

In [3]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [4]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [5]:
os.chdir("..")

In [6]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [7]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик
- категорий для дропов покупателей
- Счетов клиентов
- Внешних счетов

In [612]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0] # нужны в виде серии
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")
drop_purch_cats = pd.read_csv("./data/cleaned_data/drop_purch_cats.csv")
accounts = pd.read_csv("./data/generated_data/accounts.csv")
outer_accounts = pd.read_csv("./data/generated_data/outer_accounts.csv").iloc[:, 0] # нужны в виде серии

## Объект датакласса `DropDistributorCfg` конфиги для дропов распределителей

In [9]:
drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()

## Объект класса `DropPurchaserCfg` конфиги для дропов покупателей

In [10]:
purch_configs = drop_cfg_build.build_purch_cfg()

## Класс `DropTxnPartData` - генерация части данных транзакции
-  локация, мерчант id, ip адрес, device id, канал, тип транзакции. 

In [ ]:
class DropTxnPartData:
    """
    Класс для генерации части данных о транзакции дропа:
    канал, тип операции, мерчант, геопозиция, город, IP адрес, иногда статус.
    ------------------
    Атрибуты:
    --------
    client_info - pd.DataFrame или namedtuple. Запись с информацией о клиенте
    online_merchant_ids- pd.Series. id онлайн мерчантов
    client_devices - pd.DataFrame. Девайсы клиентов.
    last_txn - tuple. Для кэширования данных любой последней транзакции.
    """
    def __init__(self, configs: Union[DropDistributorCfg, DropPurchaserCfg]):
        """
        configs: Один из датаклассов — DropDistributorCfg или DropPurchaserCfg.
                 Содержит параметры и конфиги для генерации фрода.
        """
        self.client_info = None
        self.online_merchant_ids = configs.online_merchant_ids
        self.client_devices = configs.client_devices
        self.last_txn = None


    def assert_client_info(self):
        """
        Проверка что self.client_info не пустое
        """
        assert self.client_info is not None, \
            f"self.client_info is {type(self.client_info)}"


    def original_purchase(self, online=True, get_cached=False):
        """
        Оригинальные данные клиента для операций покупок.
        На данный момент это для дропов.
        Для операций на криптобирже и для покупки товаров дропами
        -------
        online: bool.
        get_cached: bool. Пробовать ли вернуть последние кэшированные данные
                    вместо генерации новых.
        """
        self.assert_client_info()

        if get_cached and self.last_txn is not None:
            return self.last_txn

        if online:
            merchant_id = self.online_merchant_ids.sample(n=1).iat[0]
            # Координаты города и название
            trans_lat = self.client_info.lat
            trans_lon = self.client_info.lon
            trans_ip = self.client_info.home_ip
            trans_city = self.client_info.area        
            # Семпл девайса клиента
            devices = self.client_devices.loc[self.client_devices.client_id == self.client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            txn_type = "purchase"
            # Не генерируем channel. Он должен быть определен вовне
            channel = None

            self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                            device_id, channel, txn_type
            self.last_txn = self.last_txn
            return self.last_txn

        
    def original_data(self, online, receive=None):
        """
        Получение оригинальных данных клиента для транзакции.
        Пока этот метод для клиентов дропов и, возможно, для переводов мошенникам
        ------------------------------------
        online - bool.
        received - bool.
        """
        self.assert_client_info()

        # Входящий перевод
        if online and receive:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "transfer"
            txn_type = "inbound"
            merchant_id = np.nan
            trans_lat = np.nan
            trans_lon = np.nan
            trans_city = "not applicable"

            self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                            device_id, channel, txn_type
            return self.last_txn
        
        client_info = self.client_info
        client_devices = self.client_devices
        # Исходящий перевод
        if online:
            # Для онлайна просто берется home_ip и device_id из данных клиента.
            trans_ip = client_info.home_ip
            devices = client_devices.loc[client_devices.client_id == client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            channel = "transfer"
            txn_type = "outbound"  

        # Оффлайн. Снятие в банкомате
        elif not online:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "ATM"
            txn_type = "withdrawal"
            
        merchant_id = np.nan
        # Локация транзакции просто записываем координаты и название города клиента
        trans_lat = client_info.lat
        trans_lon = client_info.lon
        trans_city = client_info.area

        self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                        device_id, channel, txn_type
        return self.last_txn
    
    
    def check_previous(self, dist, last_full):
        """
        Решить можно ли вернуть данные последней транзакции
        из кэша. True может вернуть только для дропа распределителя.
        ------
        dist: bool. Дроп распределитель или нет.
        last_full: dict. Полные данные последней транзакции.
                   Нужны, чтобы узнать channel.
        """
        if not dist:
            return False
        if last_full is None:
            return False
        if last_full["channel"] == "crypto_exchange":
            return True
        
        return False

    
    def reset_cache(self):
        """
        Сброс кэша
        """
        self.last_txn = None

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
# Временный импорт

from data_generator.fraud.txndata import DropTxnPartData
from data_generator.indev import DropConfigBuilder

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_dist_cfg()
# configs = drop_cfg_build.build_purch_cfg()
drop_clients = configs.clients
txn_part_data_test = DropTxnPartData(configs=configs)

for client in drop_clients.itertuples():
    txn_part_data_test.client_info = client
txn_part_data_test.client_info

Pandas(Index=27, client_id=1017, district_id=6, birth_date='1943-02-12', sex='female', region='Севастополь', area='Севастополь', timezone='UTC+3', lat=44.6167334, lon=33.5253552, population=344479, home_ip='2.60.3.196', geometry=<MULTIPOLYGON (((33.378 44.584, 33.379 44.585, 33.381 44.585, 33.382 44.585,...>)

In [3]:
txn_part_data_test.last_txn

**Тест для дропа распределителя** `dist=True`

In [7]:
get_cached = txn_part_data_test.check_previous(dist=True, last_full=None)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)

(np.int64(6821),
 61.2539773,
 73.3961726,
 '2.60.1.75',
 'Сургут',
 np.int64(601),
 None,
 'purchase')

In [8]:
last_full = {"channel":"crypto_exchange"}
get_cached = txn_part_data_test.check_previous(dist=True, last_full=last_full)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)
# merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type

(np.int64(6821),
 61.2539773,
 73.3961726,
 '2.60.1.75',
 'Сургут',
 np.int64(601),
 None,
 'purchase')

In [10]:
txn_part_data_test.original_data(online=False)

(nan,
 61.2539773,
 73.3961726,
 'not applicable',
 'Сургут',
 nan,
 'ATM',
 'withdrawal')

In [11]:
txn_part_data_test.last_txn

(nan,
 61.2539773,
 73.3961726,
 'not applicable',
 'Сургут',
 nan,
 'ATM',
 'withdrawal')

In [12]:
txn_part_data_test.reset_cache()
txn_part_data_test.last_txn

**Тест для дропа поукпателя** `dist=False`

In [4]:
get_cached = txn_part_data_test.check_previous(dist=False, last_full=None)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)

(np.int64(6791),
 44.6167334,
 33.5253552,
 '2.60.3.196',
 'Севастополь',
 np.int64(1730),
 None,
 'purchase')

In [6]:
last_full = {"channel":"crypto_exchange"}
get_cached = txn_part_data_test.check_previous(dist=False, last_full=last_full)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)

(np.int64(6843),
 44.6167334,
 33.5253552,
 '2.60.3.196',
 'Севастополь',
 np.int64(1730),
 None,
 'purchase')

In [7]:
txn_part_data_test.original_data(online=False)

(nan,
 44.6167334,
 33.5253552,
 'not applicable',
 'Севастополь',
 nan,
 'ATM',
 'withdrawal')

In [8]:
txn_part_data_test.last_txn

(nan,
 44.6167334,
 33.5253552,
 'not applicable',
 'Севастополь',
 nan,
 'ATM',
 'withdrawal')

## Объект класса `DropTxnPartData` для распределителей
- генерация части данных транзакции

In [92]:
part_txn_data_dist = DropTxnPartData(configs=dist_configs)

## Класс `DropBaseClasses`  
Создатель основных классов для дропов:
- DropAccountHandler
- DropAmountHandler
- DropTimeHandler
- DistBehaviorHandler или PurchBehaviorHandler
- DropTxnPartData

In [11]:
# ----------ВСТАВИТЬ ГОТОВЫЙ КЛАСС-------------------------

## Класс `CreateDropTxn`

In [270]:
class CreateDistTxn:
    """
    Создание транзакций дропа распределителя под разное поведение.
    """
    def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
                attempts=0):
        """
        timestamps - pd.DataFrame.
        trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
        drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
        in_txns - int. Количество входящих транзакций
        out_txns - int. Количество исходящих транзакций
        in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        last_txn - dict. Полные данные последней транзакции
        attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
        """
        self.timestamps = timestamps
        self.trans_partial_data = trans_partial_data
        self.drop_client = drop_client_cls
        self.in_txns = in_txns
        self.out_txns = out_txns
        self.in_lim = in_lim
        self.out_lim = out_lim
        self.last_txn = last_txn
        self.attempts = attempts


    def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False): # <------------------------- KEEP, EDIT
        """
        Один входящий/исходящий перевод либо одно снятие в банкомате.
        ---------------------
        online - bool. Онлайн перевод или снятие в банкомате.
        declined - bool. Будет ли текущая транзакция отклонена.
        in_chunks - bool. Транзакция будет частью серии транзакций.
        to_drop_share - float. Вероятность, что дроп пошлет другому дропу
        receive - входящий перевод или нет.
        """
        client_id = self.trans_partial_data.client_info.client_id # берем из namedtuple
        
        # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
        txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

        # перевод дропу
        if receive:
            self.in_txns += 1
            amount = self.drop_client.receive(declined=declined)
            account = self.drop_client.account
        # перевод от дропа    
        elif not receive and online:
            to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
            self.out_txns += 1
            account = self.drop_client.get_account(to_drop=to_drop)
        # снятие дропом    
        elif not receive and not online:
            account = self.drop_client.account
            self.out_txns += 1
        
        # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
        merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
                                                self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
        # Генерация суммы если исходящая транзакция
        # Если эта транзакция только часть серии операций для распределения всего баланса
        if in_chunks:
            chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)
            
        # Иначе если не по частям и не входящая транзакция. not receive т.к. этот метод и для входящих транзакций
        # а у входящих транзакций своя генерация суммы
        elif not in_chunks and not receive:
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

        if declined:
            status = "declined"
            is_fraud = True
            rule = "drop_flow_cashout"
        else:
            status = "approved"
            is_fraud = False
            rule = "not applicable"

        # Статичные характеристики
        is_suspicious = False
        category_name="not applicable"

        # Сборка всех данных в транзакцию и запись как послдней транзакции
        self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
                             category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
                             trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
                             is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

        return self.last_txn

    
    def limit_reached(self): # <------------------------- KEEP
        """
        Проверка достижения лимитов входящих и исходящих транзакций
        Сверка с self.in_lim и self.out_lim
        ------------------------
        Вернет True если какой либо лимит достигнут
        """
        if self.in_lim == self.in_txns:
            return True
        if self.out_lim == self.out_txns:
            return True
        return False

    def reset_cache(self, only_counters=True): # <------------------------- KEEP, EDIT
        """
        Сброос кэшированных данных
        -------------
        only_counters - bool
                        Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
                        Если False то также сбросится информация о последней транзакции self.last_txn
        """
        
        self.in_txns = 0
        self.out_txns = 0
        self.batch_txns = 0

        if only_counters:
            return

        self.last_txn = {}

**Тест `CreateDropTxn`**

In [ ]:
# import data_generator.indev
# import data_generator.fraud.drops.base
# import data_generator.configs
# import data_generator.fraud.drops.behavior
# import data_generator.fraud.txndata
# import data_generator.fraud.drops.time

# importlib.reload(data_generator.indev)
# importlib.reload(data_generator.fraud.drops.base)
# importlib.reload(data_generator.configs)
# importlib.reload(data_generator.fraud.drops.behavior)
# importlib.reload(data_generator.fraud.txndata)
# importlib.reload(data_generator.fraud.drops.time)

# acc_hand1 = DropAccountHandler(configs=configs)
# amt_hand1 = DropAmountHandler(configs=configs)
# part_data1 = DropTxnPartData(configs=configs)
# time_hand1 = DropTimeHandler(configs=configs)
# behav_hand1 = DistBehaviorHandler(configs=configs, amt_hand=amt_hand1)
# behav_hand1 = PurchBehaviorHandler(configs=configs, amt_hand=amt_hand1)

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
from data_generator.fraud.drops.base import DropAccountHandler, DropAmountHandler
from data_generator.fraud.drops.time import DropTimeHandler
# from data_generator.fraud.drops.behavior import DistBehaviorHandler, PurchBehaviorHandler
from data_generator.fraud.txndata import DropTxnPartData
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
# configs = drop_cfg_build.build_dist_cfg()
configs = drop_cfg_build.build_purch_cfg()

base_agg1 = DropBaseClasses(configs=configs)
# base_agg1.build_all(drop_type="distributor")
base_agg1.build_all(drop_type="purchaser")
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=2987, district_id=14, birth_date='1946-06-01', sex='female', region='Башкортостан', area='Стерлитамак', timezone='UTC+5', lat=53.6302498, lon=55.9315707, population=273432, home_ip='2.60.11.13', geometry=<MULTIPOLYGON (((55.883 53.635, 55.892 53.639, 55.896 53.632, 55.903 53.632,...>)

In [4]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data):
    cr_drop_txn.reset_cache()
    behav_hand.reset_cache(all=False)
    amt_hand.reset_cache(life_end=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()

In [5]:
base_agg1.part_data.client_info

Pandas(Index=8, client_id=2987, district_id=14, birth_date='1946-06-01', sex='female', region='Башкортостан', area='Стерлитамак', timezone='UTC+5', lat=53.6302498, lon=55.9315707, population=273432, home_ip='2.60.11.13', geometry=<MULTIPOLYGON (((55.883 53.635, 55.892 53.639, 55.896 53.632, 55.903 53.632,...>)

**`CreateDropTxn.trf_or_atm` тест**

In [10]:
cr_drop_txn1.in_txns, cr_drop_txn1.out_txns, cr_drop_txn1.last_txn, amt_hand1.batch_txns

(0, 0, None, 0)

In [11]:
cr_drop_txn1.txn_part_data.client_info

Pandas(Index=8, client_id=2092, district_id=72, birth_date='1945-03-21', sex='male', region='Татарстан', area='Казань', timezone='UTC+3', lat=55.7943877, lon=49.1115312, population=1216965, home_ip='2.60.7.186', geometry=<MULTIPOLYGON (((48.821 55.826, 48.821 55.826, 48.821 55.827, 48.821 55.827,...>)

**`category_and_channel`**

In [8]:
cr_drop_txn1.category_and_channel(dist=True)

('crypto_exchange', 'balance_top_up')

In [7]:
# Тест для дропа покупателя

channels1 = []
categories1 = []
chan_and_cat1 = pd.DataFrame()

for _ in range(1000):
    chan1, cat1 = cr_drop_txn1.category_and_channel(dist=False)
    channels1.append(chan1)
    categories1.append(cat1)
    
chan_and_cat1["channel"] = channels1
chan_and_cat1["category"] = categories1

In [14]:
chan_and_cat1.value_counts(normalize=True).reset_index()

,channel,category,proportion
0,ecom,shopping_net,0.570
1,ecom,misc_net,0.262
2,ecom,travel_net,0.168


In [5]:
categories = configs.categories
categories

,category,weight
0,shopping_net,0.60
1,misc_net,0.25
2,travel_net,0.15


**`status_and_rule`**  
значения: status, is_fraud, rule

In [10]:
cr_drop_txn1.status_and_rule(declined=True, dist=True)

('declined', True, 'drop_flow_cashout')

### **`trf_or_atm`**

НЕ отклоненный входящий перевод  
`BehavHand.online=None, receive=True, declined=False, BehavHand.in_chunks=False`

In [12]:
acc_hand1.accounts.query("client_id == 11794")["account_id"]
acc_hand1.accounts.loc[acc_hand1.accounts.client_id == acc_hand1.client_id, "account_id"].iat[0]

np.int64(13672)

In [11]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.online = None
behav_hand1.in_chunks = None
receive_txn = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

pd.DataFrame([receive_txn])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-09 22:20:00,1736461200,32100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable


In [12]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(32100.0), 0)

In [13]:
cr_drop_txn1.in_txns, cr_drop_txn1.out_txns

(1, 0)

НЕ отклоненный исходящий перевод целиком  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=False`

In [14]:
part_data1.client_info.area

'Стерлитамак'

In [15]:
behav_hand1.online = True
behav_hand1.in_chunks = False

whole_out = cr_drop_txn1.trf_or_atm(receive=False, to_drop=False, declined=False)
pd.concat([pd.DataFrame([receive_txn]), pd.DataFrame([whole_out])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-09 22:20:00,1736461200,32100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-10 00:43:00,1736469780,32100.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,19221,False,False,approved,not applicable


НЕ отклоненное снятие целиком  
`BehavHand.online=False, receive=False, declined=False, BehavHand.in_chunks=False, BehavHand.scen = "atm"`

In [16]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm"
behav_hand1.guide_scenario()
behav_hand1.in_chunks = False

receive_txn2 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
whole_atm = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)

pd.concat([pd.DataFrame([receive_txn2]), pd.DataFrame([whole_atm])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-27 13:19:00,1737983940,19800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-27 15:58:00,1737993480,19800.0,withdrawal,ATM,not applicable,False,NaN,Стерлитамак,53.63025,55.931571,not applicable,NaN,12828,False,False,approved,not applicable


In [17]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 1)

НЕ отклоненный исходящий перевод частями  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [18]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks = True

receive_txn3 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

all_txns3 = [receive_txn3]
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)
    all_txns3.append(part_out)
pd.DataFrame(all_txns3)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-21 02:30:00,1737426600,32000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-21 03:34:00,1737430440,29000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,17352,False,False,approved,not applicable
2,2987,2025-01-21 04:04:00,1737432240,3000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,15859,False,False,approved,not applicable


In [19]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 2)

НЕ отклоненное снятие и перевод(ы) частями

In [20]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm+transfer"
behav_hand1.in_chunks = True

receive_txn4 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns4 = [receive_txn4]

while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)
    all_txns4.append(part_out)
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-16 23:34:00,1737070440,49700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-17 00:45:00,1737074700,20800.0,withdrawal,ATM,not applicable,False,NaN,Стерлитамак,53.63025,55.931571,not applicable,NaN,12828,False,False,approved,not applicable
2,2987,2025-01-17 02:25:00,1737080700,15000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,15388,False,False,approved,not applicable
3,2987,2025-01-17 04:45:00,1737089100,12000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,17529,False,False,approved,not applicable
4,2987,2025-01-17 05:41:00,1737092460,1900.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,17449,False,False,approved,not applicable


In [21]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 4)

Отклоненный входящий перевод  
`BehavHand.online=None, receive=True, declined=True, BehavHand.in_chunks=False`

In [22]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.online = None
behav_hand1.in_chunks = None
receive_txn5 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=True)

pd.DataFrame([receive_txn5])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-01 14:45:00,1735742700,44500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,True,False,declined,drop_flow_cashout


In [23]:
amt_hand1.balance, amt_hand1.batch_txns

(0, 0)

Отклоненный исходящий перевод целиком  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=False`

In [24]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "transfer"
behav_hand1.in_chunks = False

receive_txn4 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns4 = [receive_txn4]
i = 0
while amt_hand1.balance > 0 and i < 4:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns4.append(part_out)
    i += 1
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-06 00:14:00,1736122440,57100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-06 02:39:00,1736131140,57100.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,18369,True,False,declined,drop_flow_cashout
2,2987,2025-01-06 04:31:00,1736137860,42900.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,20609,True,False,declined,drop_flow_cashout
3,2987,2025-01-06 05:30:00,1736141400,28700.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,16608,True,False,declined,drop_flow_cashout
4,2987,2025-01-06 06:36:00,1736145360,14500.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,25233,True,False,declined,drop_flow_cashout


In [25]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(57100.0), 4, 4)

**Отклоненное снятие целиком**  
`BehavHand.online=False, receive=False, declined=True, BehavHand.in_chunks=False`

In [26]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm"
behav_hand1.in_chunks = False

receive_txn7 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

behav_hand1.guide_scenario()
whole_out7 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
pd.DataFrame([receive_txn7, whole_out7])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-26 09:32:00,1737883920,12900.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-26 11:12:00,1737889920,12900.0,withdrawal,ATM,not applicable,False,NaN,Стерлитамак,53.63025,55.931571,not applicable,NaN,12828,True,False,declined,drop_flow_cashout


In [27]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(12900.0), 1, 1)

**Отклоненный  исходящий перевод частями**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=True`

In [28]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks = True

receive_txn8 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns8 = [receive_txn8]
i = 0
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out8 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns8.append(part_out8)
    i += 1
    if i > 4:
        break
        
pd.DataFrame(all_txns8)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-17 16:52:00,1737132720,42100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-17 19:27:00,1737142020,16000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,16705,True,False,declined,drop_flow_cashout
2,2987,2025-01-17 19:58:00,1737143880,12000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,20762,True,False,declined,drop_flow_cashout
3,2987,2025-01-17 20:36:00,1737146160,8000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,22157,True,False,declined,drop_flow_cashout
4,2987,2025-01-17 23:21:00,1737156060,4000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,24942,True,False,declined,drop_flow_cashout
5,2987,2025-01-18 00:17:00,1737159420,3000.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,23067,True,False,declined,drop_flow_cashout


In [29]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(42100.0), 5, 5)

**Отклоненное снятие частями**  
`BehavHand.online=False, receive=False, declined=True, BehavHand.in_chunks=True`

In [30]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm+transfer"
behav_hand1.in_chunks = True

receive_txn9 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns9 = [receive_txn9]
i = 0
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out9 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns9.append(part_out9)
    i += 1
    if i > 3:
        break
        
pd.DataFrame(all_txns9)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-29 11:23:00,1738149780,37500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-29 13:15:00,1738156500,20900.0,withdrawal,ATM,not applicable,False,NaN,Стерлитамак,53.63025,55.931571,not applicable,NaN,12828,True,False,declined,drop_flow_cashout
2,2987,2025-01-29 13:46:00,1738158360,15700.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,17784,True,False,declined,drop_flow_cashout
3,2987,2025-01-29 16:29:00,1738168140,10500.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,18856,True,False,declined,drop_flow_cashout
4,2987,2025-01-29 17:17:00,1738171020,5300.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,19421,True,False,declined,drop_flow_cashout


**НЕ отклоненный перевод другому дропу**  
Дропов >= `AccHand.min_drops`

In [31]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


6

In [32]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn10 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns10 = [receive_txn10]


behav_hand1.guide_scenario()
part_out10 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns10.append(part_out10)
all_df10 = pd.DataFrame(all_txns10)
all_df10

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-23 09:57:00,1737626220,54300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-23 11:58:00,1737633480,54300.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,13580,False,False,approved,not applicable


In [33]:
target_acc10 = all_df10.loc[1, "account"]
acc_hand1.accounts.query("account_id == @target_acc10")

,client_id,account_id,is_drop
3580,3782,13580,True


**НЕ отклоненный перевод другому дропу**  
Дропов < `AccHand.min_drops`

In [34]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops - 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [35]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn11 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns11 = [receive_txn11]


behav_hand1.guide_scenario()
part_out11 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns11.append(part_out11)
all_df11 = pd.DataFrame(all_txns11)
all_df11

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-28 17:38:00,1738085880,17900.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-28 18:24:00,1738088640,17900.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,19148,False,False,approved,not applicable


In [37]:
target_acc11 = all_df11.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc11]

3779    19148
Name: account_id, dtype: int64

**НЕ отклоненный перевод другому дропу**  
Дропов НЕТ

In [38]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


0

In [39]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn12 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns12 = [receive_txn12]

behav_hand1.guide_scenario()
part_out12 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns12.append(part_out12)
all_df12 = pd.DataFrame(all_txns12)
all_df12

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-29 14:32:00,1738161120,37300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-29 16:05:00,1738166700,37300.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,16224,False,False,approved,not applicable


In [40]:
target_acc12 = all_df12.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc12]

855    16224
Name: account_id, dtype: int64

**Отклоненный перевод другому дропу**  
Дропов >= AccHand.min_drops

In [41]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops + 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


7

In [42]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn13 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns13 = [receive_txn13]

behav_hand1.guide_scenario()
part_out13 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns13.append(part_out13)
all_df13 = pd.DataFrame(all_txns13)
all_df13

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-05 18:44:00,1736102640,53700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-05 20:24:00,1736108640,53700.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,12619,True,False,declined,drop_flow_cashout


In [43]:
target_acc13 = all_df13.loc[1, "account"]
acc_hand1.accounts.query("account_id == @target_acc13")

,client_id,account_id,is_drop
2619,2771,12619,True


**Отклоненный перевод другому дропу**  
Дропов < AccHand.min_drops

In [44]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops - 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [45]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn14 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns14 = [receive_txn14]

behav_hand1.guide_scenario()
part_out14 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns14.append(part_out14)
all_df14 = pd.DataFrame(all_txns14)
all_df14

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-11 11:35:00,1736595300,40300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828,False,False,approved,not applicable
1,2987,2025-01-11 13:09:00,1736600940,40300.0,outbound,transfer,not applicable,True,NaN,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,19522,True,False,declined,drop_flow_cashout


In [46]:
target_acc14 = all_df14.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc14]

4153    19522
Name: account_id, dtype: int64

**Отклоненный перевод другому дропу**  
Дропов НЕТ

In [57]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


0

In [58]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn15 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns15 = [receive_txn15]

behav_hand1.guide_scenario()
part_out15 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns15.append(part_out15)
all_df15 = pd.DataFrame(all_txns15)
all_df15

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3881,2025-01-17 16:32:00,1737131520,15300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13672,False,False,approved,not applicable
1,3881,2025-01-17 18:43:00,1737139380,15300.0,outbound,transfer,not applicable,True,NaN,Курган,55.444345,65.316134,2.60.14.89,6620.0,19855,True,False,declined,drop_flow_cashout


In [59]:
target_acc15 = all_df15.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc15]

4486    19855
Name: account_id, dtype: int64

**Доля переводов дропам**  
Дропов `>=` AccHand.min_drops. Проверка в цикле

In [60]:
own_id = acc_hand1.client_id
min_drops = configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops + 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


7

In [63]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "split_transfer"
all_txns16 = []

i = 0
while i < 300:
    receive_txn16 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
    all_txns16.append(receive_txn16)
    

    while amt_hand1.balance > 0:
        behav_hand1.in_chunks_val()
        behav_hand1.guide_scenario()
        to_drop16 = behav_hand1.to_drop
        part_out16 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=to_drop16, receive=False)
        all_txns16.append(part_out16)
    i += 1
    
all_df16 = pd.DataFrame(all_txns16)
print(f"{all_df16.shape[0]} rows")
all_df16.head()

1295 rows


,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3881,2025-01-13 12:17:00,1736770620,26000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13672,False,False,approved,not applicable
1,3881,2025-01-13 12:58:00,1736773080,21000.0,outbound,transfer,not applicable,True,NaN,Курган,55.444345,65.316134,2.60.14.89,6620.0,18754,False,False,approved,not applicable
2,3881,2025-01-13 14:10:00,1736777400,5000.0,outbound,transfer,not applicable,True,NaN,Курган,55.444345,65.316134,2.60.14.89,6620.0,13334,False,False,approved,not applicable
3,3881,2025-01-13 17:10:00,1736788200,33200.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13672,False,False,approved,not applicable
4,3881,2025-01-14 15:02:00,1736866920,14000.0,outbound,transfer,not applicable,True,NaN,Курган,55.444345,65.316134,2.60.14.89,6620.0,23958,False,False,approved,not applicable


In [64]:
own_id16 = acc_hand1.client_id
drop_accounts16 = acc_hand1.accounts.query("client_id != @own_id16 and is_drop == True")
df16_merged = all_df16.merge(drop_accounts16, how="outer", left_on="account", right_on="account_id").sort_values("unix_time")
df16_no_rcvd = df16_merged.query("type != 'inbound'")
df16_no_rcvd.head()

,client_id_x,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,...,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule,client_id_y,account_id,is_drop
696,3881,2025-01-13 12:58:00,1736773080,21000.0,outbound,transfer,not applicable,True,NaN,Курган,...,2.60.14.89,6620.0,18754,False,False,approved,not applicable,NaN,NaN,NaN
45,3881,2025-01-13 14:10:00,1736777400,5000.0,outbound,transfer,not applicable,True,NaN,Курган,...,2.60.14.89,6620.0,13334,False,False,approved,not applicable,3519.0,13334.0,True
1167,3881,2025-01-14 15:02:00,1736866920,14000.0,outbound,transfer,not applicable,True,NaN,Курган,...,2.60.14.89,6620.0,23958,False,False,approved,not applicable,NaN,NaN,NaN
723,3881,2025-01-14 15:45:00,1736869500,14000.0,outbound,transfer,not applicable,True,NaN,Курган,...,2.60.14.89,6620.0,19004,False,False,approved,not applicable,NaN,NaN,NaN
874,3881,2025-01-14 17:34:00,1736876040,5000.0,outbound,transfer,not applicable,True,NaN,Курган,...,2.60.14.89,6621.0,20913,False,False,approved,not applicable,NaN,NaN,NaN


In [65]:
df16_no_rcvd.is_drop.value_counts(normalize=True, dropna=False)

is_drop
NaN     0.905528
True    0.094472
Name: proportion, dtype: float64

**Отклоненный  исходящий перевод частями и заодно набросок полного цикла дропа распределителя**

In [1794]:
# Объявление новых объектов классов для тестов.
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)


drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)
create_drop_txn_tst.reset_cache(only_counters=False)
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, drop_client_test2.batch_txns, create_drop_txn_tst.limit_reached()

(0, 0, 0, False)

In [ ]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
all_txns5 = []
declined=False
create_drop_txn_tst.attempts_after_decline(min=0, max=1)
drop_client_test2.chunk_size = 5000
drop_client_test2.batch_txns = 1

i = 1

while True:
    receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
    all_txns5.append(receive_txn8)
    if declined:
        break
        
    while drop_client_test2.balance > 0:
        part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
        all_txns5.append(declined)
        print(f"iter {i}")
        create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
        if create_drop_txn_tst.stop_after_decline(declined=declined):
            break
           
        declined = create_drop_txn_tst.limit_reached()
        i += 1
        
    # pd.DataFrame(all_txns5)

Ниже нерабочий код т.к. это прикидки с криптой

In [ ]:
# create_drop_txn_tst.reset_cache(only_counters=False)
# drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
# all_txns5 = []
# declined=False
# crypto_rate = 0.05
# create_drop_txn_tst.attempts_after_decline(min=0, max=1)
# drop_client_test2.chunk_size = 5000
# drop_client_test2.batch_txns = 1

# i = 1

# while True:
#     receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
#     all_txns5.append(receive_txn8)
#     if declined:
#         break
        
#     while drop_client_test2.balance > 0:
#         if np.random.uniform(0, 1) < crypto_rate:
#             # использовать метод FraudTransPartialData.original_purchase внутри
#             part_out = create_drop_txn_tst.purchase(online=True, declined=declined, in_chunks=True, channel="crypto_exchange")
#         else:
#             part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
#         all_txns5.append(part_out)
#         print(f"iter {i}")
#         create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
#         if create_drop_txn_tst.stop_after_decline(declined=declined):
#             break
           
#         declined = create_drop_txn_tst.limit_reached()
#         i += 1
        
#     # pd.DataFrame(all_txns5)

## **Тест `CreateTxn.purchase`**

### **`dist=True` дроп распределитель**

**НЕ отклоненная покупка крипты целиком**  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=False`

In [47]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn17 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns17 = [receive_txn17]

behav_hand1.guide_scenario()
whole_out17 = cr_drop_txn1.purchase(declined=False, dist=True)
all_txns17.append(whole_out17)
all_df17 = pd.DataFrame(all_txns17)
all_df17

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-07 20:19:00,1736281140,36700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828.0,False,False,approved,not applicable
1,2987,2025-01-07 21:36:00,1736285760,36700.0,purchase,crypto_exchange,balance_top_up,True,6825.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable


In [48]:
amt_hand1.batch_txns, time_hand1.in_txns, time_hand1.out_txns

(1, 1, 1)

**НЕ отклоненная покупка крипты частями**  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [49]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns18 = []

receive_txn18 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns18.append(receive_txn18)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks_val()


while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out18 = cr_drop_txn1.purchase(declined=False, dist=True)
    all_txns18.append(part_out18)
    
all_df18 = pd.DataFrame(all_txns18)

In [50]:
all_df18

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-04 19:40:00,1736019600,46300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828.0,False,False,approved,not applicable
1,2987,2025-01-04 20:56:00,1736024160,21000.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable
2,2987,2025-01-04 21:52:00,1736027520,9000.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable
3,2987,2025-01-04 22:36:00,1736030160,6000.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable
4,2987,2025-01-05 00:31:00,1736037060,7000.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable
5,2987,2025-01-05 01:06:00,1736039160,3000.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable
6,2987,2025-01-05 20:08:00,1736107680,300.0,purchase,crypto_exchange,balance_top_up,True,6816.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5103.0,NaN,False,False,approved,not applicable


**Отклоненная покупка крипты целиком**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=False`

In [51]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns19 = []

receive_txn19 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns19.append(receive_txn19)

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

i = 0
while amt_hand1.balance > 0 and i < 3:
    behav_hand1.guide_scenario()
    part_out19 = cr_drop_txn1.purchase(declined=True, dist=True)
    all_txns19.append(part_out19)
    i += 1
    
all_df19 = pd.DataFrame(all_txns19)
all_df19

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-30 03:51:00,1738209060,26300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828.0,False,False,approved,not applicable
1,2987,2025-01-30 06:45:00,1738219500,26300.0,purchase,crypto_exchange,balance_top_up,True,6859.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout
2,2987,2025-01-30 09:26:00,1738229160,19800.0,purchase,crypto_exchange,balance_top_up,True,6859.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout
3,2987,2025-01-30 10:15:00,1738232100,13300.0,purchase,crypto_exchange,balance_top_up,True,6859.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout


**Отклоненная  покупка крипты частями**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=True`

In [52]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns20 = []

receive_txn20 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns20.append(receive_txn20)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks_val()

i = 0
while amt_hand1.balance > 0 and i < 4:
    behav_hand1.guide_scenario()
    part_out20 = cr_drop_txn1.purchase(declined=True, dist=True)
    all_txns20.append(part_out20)
    i += 1
    
all_df20 = pd.DataFrame(all_txns20)
all_df20

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2987,2025-01-29 01:14:00,1738113240,37200.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12828.0,False,False,approved,not applicable
1,2987,2025-01-29 03:06:00,1738119960,20000.0,purchase,crypto_exchange,balance_top_up,True,6955.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout
2,2987,2025-01-29 04:40:00,1738125600,15000.0,purchase,crypto_exchange,balance_top_up,True,6955.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout
3,2987,2025-01-29 06:09:00,1738130940,10000.0,purchase,crypto_exchange,balance_top_up,True,6955.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout
4,2987,2025-01-29 06:56:00,1738133760,5000.0,purchase,crypto_exchange,balance_top_up,True,6955.0,Стерлитамак,53.63025,55.931571,2.60.11.13,5102.0,NaN,True,False,declined,drop_flow_cashout


In [53]:
amt_hand1.balance

np.float64(37200.0)

### **`dist=False` дроп покупатель**

**НЕ отклоненная покупка целиком**  
`BehavHand.online=True, declined=False, BehavHand.in_chunks=False`

In [4]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns21 = []

receive_txn21 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns21.append(receive_txn21)

behav_hand1.in_chunks = False

while amt_hand1.balance > 0:
    part_out21 = cr_drop_txn1.purchase(declined=False, dist=False)
    all_txns21.append(part_out21)
    
all_df21 = pd.DataFrame(all_txns21)
all_df21

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11034,2025-01-17 04:31:00,1737088260,23800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15134.0,False,False,approved,not applicable
1,11034,2025-01-17 07:05:00,1737097500,23800.0,purchase,ecom,shopping_net,True,6898.0,Москва,55.753879,37.620373,2.60.20.15,9232.0,NaN,False,False,approved,not applicable


**НЕ отклоненная покупка частями**  
`BehavHand.online=True, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [5]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns22 = []

receive_txn22 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns22.append(receive_txn22)

behav_hand1.in_chunks = True

while amt_hand1.balance > 0:
    part_out22 = cr_drop_txn1.purchase(declined=False, dist=False)
    all_txns22.append(part_out22)
    
all_df22 = pd.DataFrame(all_txns22)
all_df22

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11034,2025-01-20 14:47:00,1737384420,21100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15134.0,False,False,approved,not applicable
1,11034,2025-01-20 16:48:00,1737391680,19000.0,purchase,ecom,travel_net,True,6831.0,Москва,55.753879,37.620373,2.60.20.15,9232.0,NaN,False,False,approved,not applicable
2,11034,2025-01-20 18:09:00,1737396540,2100.0,purchase,ecom,misc_net,True,6842.0,Москва,55.753879,37.620373,2.60.20.15,9233.0,NaN,False,False,approved,not applicable


**Отклоненная покупка целиком**  
`BehavHand.online=True, declined=True, BehavHand.in_chunks=False`

In [21]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns23 = []

receive_txn23 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns23.append(receive_txn23)

behav_hand1.online = True
behav_hand1.in_chunks = False

i = 0
while amt_hand1.balance > 0 and i < 4:
    part_out23 = cr_drop_txn1.purchase(declined=True, dist=False)
    all_txns23.append(part_out23)
    i += 1
    
all_df23 = pd.DataFrame(all_txns23)
all_df23

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,676,2025-01-07 19:25:00,1736277900,20800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,10644.0,False,False,approved,not applicable
1,676,2025-01-07 21:26:00,1736285160,20800.0,purchase,ecom,shopping_net,True,6788.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
2,676,2025-01-07 23:02:00,1736290920,15600.0,purchase,ecom,misc_net,True,6803.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
3,676,2025-01-08 01:15:00,1736298900,10400.0,purchase,ecom,shopping_net,True,6854.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
4,676,2025-01-08 03:46:00,1736307960,5200.0,purchase,ecom,misc_net,True,6875.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser


In [22]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 5, step=1)
cum_sub_df["amt"] = - (20800 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 20800
cum_sub_df.amt.cumsum()

0    20800.0
1    15600.0
2    10400.0
3     5200.0
4        0.0
Name: amt, dtype: float64

**Отклоненная  покупка частями**  
`BehavHand.online=True, declined=True, BehavHand.in_chunks=True`

In [23]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns24 = []

receive_txn24 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns24.append(receive_txn24)

behav_hand1.online = True
behav_hand1.in_chunks = True

i = 0
while amt_hand1.balance > 0 and i < 4:
    part_out24 = cr_drop_txn1.purchase(declined=True, dist=False)
    all_txns24.append(part_out24)
    i += 1
    
all_df24 = pd.DataFrame(all_txns24)
all_df24

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,676,2025-01-06 06:37:00,1736145420,6100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,10644.0,False,False,approved,not applicable
1,676,2025-01-06 07:35:00,1736148900,6000.0,purchase,ecom,shopping_net,True,6814.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
2,676,2025-01-06 10:22:00,1736158920,4500.0,purchase,ecom,misc_net,True,6823.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
3,676,2025-01-06 12:31:00,1736166660,3000.0,purchase,ecom,shopping_net,True,6777.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser
4,676,2025-01-06 14:26:00,1736173560,3000.0,purchase,ecom,shopping_net,True,6815.0,Новосибирск,55.028102,82.921058,2.60.2.133,1162.0,NaN,True,False,declined,drop_purchaser


In [24]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 5, step=1)
cum_sub_df["amt"] = - (6000.0 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 6000.0
cum_sub_df.amt.cumsum()

0    6000.0
1    4500.0
2    3000.0
3    1500.0
4       0.0
Name: amt, dtype: float64

**`limit_reached`**

In [28]:
cr_drop_txn1.reset_cache()
cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
cr_drop_txn1.limit_reached()

True

**Конец теста `CreateDropTxn`**

# Конец информативной части

# Старый код, функции и т.д.

## Класс `DropClient`
- метод get_account(self): Мб добавить вариант to_drop: bool. Перевод другому имеющемуся дропу. при этом. 
    - записываем счет дропа в свой счет. Атрибут account
    - записываем  счет дропа в атрибут drop_accounts
    - если надо перевести другому дропу, то фильтруем drop_accounts исключая свой
    - надо тогда клиентам приписать шестизначные счета. просто по порядку.
    - счета куда отправляют дропы это номера начинающиеся со счета последнего клиента + 1

In [ ]:
# class DropClient: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий, номеров счетов.
#     Управление балансом текущего дропа.
#     """
#     def __init__(self, accounts, account, outer_accounts, balance=0, batch_txns=0, chunk_size=0, used_accounts=pd.Series(name="account_id")):
#         """
#         accounts - pd.DataFrame. Счета клиентов банка.
#         account - int. Номер счета текущего дропа.
#         outer_accounts - pd.Series. Номера счетов для входящих и исходящих переводов в/из других банков.
#         balance - float. Текущий баланс дропа
#         batch_txns - int. Счетчик транзакций сделанных в рамках распределения полученной партии денег
#         chunk_size - int, float. Последний созданный размер части баланса для перевода по частям.
#         used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
#         """
#         self.accounts = accounts
#         self.account = account
#         self.outer_accounts = outer_accounts
#         self.balance = balance
#         self.batch_txns = batch_txns
#         self.chunk_size = chunk_size
#         self.used_accounts = used_accounts

#     def update_balance(self, amount, add=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         add - bool. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if add:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined, low=5000, high=100000, mean=30000, std=20000, round=500):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         low - float. Минимальная сумма
#         high - float. Максимальная сумма
#         mean - float. Средняя сумма
#         std - float. Стандартное отклонение
#         round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце
#                      При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1
#         """

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // round * round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, add=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False, atm_min=10000, atm_share=0.5, round=500, rand_rate=0.9, start=0, stop=0, step=0):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля atm_share от баланса, но не меньше atm_min
#         atm_min - int, float. Минимальная сумма снятия дропом в банкомате.
#         atm_share - float. Доля от баланса если снятие через банкомат.
#         round - int. Округление целой части. По умолчанию 500. 
#                      Значит суммы будут округлены до тысяч или пяти сотен
#         rand_rate - float. От 0 до 1. Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                            Но возможны случайные совпадаения с предыдущим размером чанка.
#                            Доля случайных размеров подряд будет:
#                            p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                            p(rr) - rand_rate
#                            p(rr) - (p(rr) * p(c)). Например p(rr) = 0.9, 5 вариантов размеров чанка - p(c) = 0.20
#                            0.9 - (0.9 * 0.2) = 0.72
#                            В около 72% случаев размеры чанков не будут подряд одинаковыми 
#         start - int. Минимальный размер. Прописываем если генерация не через share.
#                      То же самое для stop и step
#         stop - int. Максимальный размер - не входит в возможный выбор.
#                     Максимальное генерируемое значение равно stop - step
#         step - int. Шаг размеров.
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если перевод
#         if online:
#             sampling_array = np.arange(start, stop, step)
#             self.chunk_size = np.random.choice(sampling_array)
#             return self.chunk_size
            
#         # Если снятие    
#         self.chunk_size = max(atm_min, self.balance * atm_share // round * round)
#         return self.chunk_size
            
        
#     def one_operation(self, amount=0, declined=False, in_chunks=False):
#         """
#         Генерация суммы операции дропа.
#         ---------
#         amount - float, int. Сумма перевода если перевод по частям - in_chunks == True
#         declined - bool. Отклонена ли транзакция или одобрена
#         in_chunks - bool. Перевод по частям или целиком. Если False, то просто пробуем перевести все с баланса
#                           При True нужно указать amount.
#         """
#         if in_chunks and amount <= 0:
#             raise ValueError(f"""If in_chunks is True, then amount must be greater than 0.
# Passed amount: {amount}""")

#         # Если перевод не по частям. Пробуем перевестит все с баланса. 
#         if not in_chunks:
#             amount = self.balance
#             self.update_balance(amount=self.balance, add=False, declined=declined)
#             # Прибавляем счетчик транзакции для текущей партии денег
#             self.batch_txns += 1
#             return amount

#         # Иначе считаем сколько частей исходя из размера одной части
#         chunks = self.balance // amount

#         # Если целое число частей больше 0. Пробуем перевести одну часть
#         if chunks > 0:
#             self.update_balance(amount=amount, add=False, declined=declined)
#             self.batch_txns += 1
#             return amount

#         # Если баланс меньше одной части. Пробуем перевести то что осталось
#         rest = self.balance
#         self.update_balance(amount=rest, add=False, declined=declined)
#         self.batch_txns += 1
#         return rest

#     def get_account(self, to_drop):
#         """
#         Номер счета входящего/исходящего перевода
#         to_drop - bool. Перевод другому дропу в нашем банке или нет.
#         """
#         # Фильтруем accounts исключая свой счет и выбирая дропов. Для случая если to_drop
#         drop_accounts = self.accounts.loc[(self.accounts.account_id != self.account) & (self.accounts.is_drop == True)]

#         # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
#         if to_drop and not drop_accounts.empty: 
#             account = drop_accounts.account_id.sample(1).iat[0]
#             # Добавляем этот счет в использованные как последнюю запись в серии
#             self.used_accounts.loc[self.used_accounts.shape[0]] = account
#             return account

#         # Если отправляем/получаем из другого банка.  
#         # Семплируем номер внешнего счета который еще не использовался
#         account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
#         # Добавляем этот счет в использованные как последнюю запись в серии
#         self.used_accounts.loc[self.used_accounts.shape[0]] = account
        
#         return account

#     def reset_cache(self, balance=True, used_accounts=True, chunk_size=True, batch_txns=True):
#         """
#         Сброс кэшированных значений
#         По умолчанию сбрасывается всё. Если что-то надо оставить, то надо выставить False
#         для этого
#         -----------------
#         balance - bool
#         used_accounts - bool
#         chunk_size - bool
#         batch_txns - bool
#         """
#         if balance:
#             self.balance = 0
#         if used_accounts:
#             self.used_accounts = pd.Series(name="account_id")
#         if chunk_size:
#             self.chunk_size = 0
#         if batch_txns:
#             self.batch_txns = 0